In [1]:
%load_ext autoreload 
%autoreload 2

from imports import *
import queue_system as qs
from queue_system import *

seed = random.seed(10)
seed_np = np.random.seed(10)

# Aggregation

In [7]:
with open('Q/Q_start.pkl', "rb") as fp:
    Q_start = pickle.load(fp)

In [4]:
weights_dict = {}
model_agg = Aggregation(graph, arrival_rates, experts, eta, queue_max)
model = StochasticMatching(graph, arrival_rates, queue_max)

state_space = list(itertools.product(np.array(range(queue_max + 1)), repeat=len(graph.keys())))
num_states = len(state_space)
n=0
state_space_ind = {}
for s in state_space:
    state_space_ind[str(n)] = s
    n += 1 
    
state_space_dict = {s : 0 for s in state_space}
len_updates = 50
res = int(len_updates / 10)
num_repeat_est = 10

queues_lenght = {v: [] for v in verteces}

K = len(experts)
weights_dict['-1'] = {state: np.array([1/K] * K) for state in state_space_ind.keys()}

repeat_weights = 10
learning_rate = 0.1# try even bigger
# learning_rate = 0.00014

In [5]:
%%time 
random.seed(6)
np.random.seed(6)
for n in range(repeat_weights):
    weights_dict = {}
    model_agg = Aggregation(graph, arrival_rates, experts, eta, queue_max, Q_start)
    model = StochasticMatching(graph, arrival_rates, queue_max)
    weights_dict['-1'] =  np.ones([num_states, K]) / K 
    time_expw = []
    time_tot = 0
    for i in tqdm(range(len_updates + 1)):
        if (i <= 200) and (i%10 == 0):
            start_time = time.time()
            weights = model_agg.aggregation_update_exp_NN(discount, learning_rate, model, H, k, num_repeat_est)
            end_time = time.time()
            time_tot += end_time - start_time
            time_expw.append(time_tot)
            weights_dict[str(i)] = weights.copy()
            random_state = state_space[random.randint(0, len(state_space)-1)]

        elif (i%res == 0) and (i>200):
            start_time = time.time()
            weights = model_agg.aggregation_update_exp_NN(discount, learning_rate, model, H, k, num_repeat_est)
            end_time = time.time()
            time_tot += end_time - start_time
            time_expw.append(time_tot)
            weights_dict[str(i)] = weights.copy()
            random_state = state_space[random.randint(0, len(state_space)-1)]

        else:
            start_time = time.time()
            weights = model_agg.aggregation_update_exp_NN(discount, learning_rate, model, H, k, num_repeat_est)
            end_time = time.time()
            time_tot += end_time - start_time
    with open('weights/weights_expw_NN8' + str(n) + '.pkl', 'wb') as output:
        pickle.dump(weights_dict, output)       

 12%|█████▏                                      | 6/51 [01:55<14:23, 19.19s/it]


KeyboardInterrupt: 

In [6]:
model_agg.A[1000:] 

array([[-0.02617405,  0.26081984, -0.19188003,  0.3918881 ,  0.06456553],
       [-0.02614339,  0.27372025, -0.19324704,  0.3972993 ,  0.06600071],
       [-0.03959581,  0.06945378, -0.15779023,  0.42669355,  0.09117832],
       ...,
       [-0.05470851,  0.20333424, -0.17474079,  0.38328689,  0.07753256],
       [-0.07128723,  0.2573014 , -0.17571674,  0.38169052,  0.09616135],
       [-0.08378263,  0.29267555, -0.17369632,  0.37698184,  0.11036169]])